<a href="https://colab.research.google.com/github/YassineOurara/illness-recognition-NLP/blob/main/Projet_YassineOurara_RecM_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import pandas as pd
import numpy as np
from tabulate import tabulate
import collections
from sklearn.feature_extraction.text import CountVectorizer
from string import punctuation
import nltk
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('omw-1.4')
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.util import ngrams
from nltk.tokenize import sent_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
en_stopwords = stopwords.words('english')
from textblob import TextBlob
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

import re
from sklearn.preprocessing import LabelEncoder

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam
! pip install wordcloud
from wordcloud import WordCloud
sns.set_palette('Set2')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


**Loading the data**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = '/content/drive/MyDrive/NLP-project/Dataset/Symptom2Disease.csv'

In [ ]:
data = pd.read_csv(path)
data.shape

(1200, 3)

In [ ]:
data.head(3)

,Unnamed: 0,label,text
0,0,Psoriasis,I have been experiencing a skin rash on my arm...
1,1,Psoriasis,"My skin has been peeling, especially on my kne..."
2,2,Psoriasis,I have been experiencing joint pain in my fing...


In [ ]:
data['label'].value_counts()

Psoriasis                          50
Varicose Veins                     50
peptic ulcer disease               50
drug reaction                      50
gastroesophageal reflux disease    50
allergy                            50
urinary tract infection            50
Malaria                            50
Jaundice                           50
Cervical spondylosis               50
Migraine                           50
Hypertension                       50
Bronchial Asthma                   50
Acne                               50
Arthritis                          50
Dimorphic Hemorrhoids              50
Pneumonia                          50
Common Cold                        50
Fungal infection                   50
Dengue                             50
Impetigo                           50
Chicken pox                        50
Typhoid                            50
diabetes                           50
Name: label, dtype: int64

In [ ]:
len(data['label'].value_counts())

24

**Preprocessing the data**

In [ ]:
# Les fonctions pour le prétraitement
def supp_ponc(text):
    txt= []
    for word in text:
        if word not in punctuation:
            txt.append(word)
    return txt
def larger_tokens(text):
    result=[]
    for word in text:
        if(len(word)>3):
            result.append(word)
    return result
def remove_stopwords(text):
    result = []
    for token in text:
        if token not in en_stopwords:
            result.append(token)           
    return result

def stemming(text):
    porter = PorterStemmer()
    
    result=[]
    for word in text:
        result.append(porter.stem(word))
    return result

def lemmatizing(text):
    lemmatizer = WordNetLemmatizer()
    
    result=[]
    for word in text:
        result.append(lemmatizer.lemmatize(word))
    return result



def clean(Dt):
    res =[]
    for text in Dt:
        res.append(' '.join(text))
    return res

In [ ]:
liste_tok=[]
for msg in data['text']:
    liste_tok.append(supp_ponc(word_tokenize(msg)))


tokens_large_liste = []
for i in range (len(liste_tok)):
    tokens_large_liste.append(larger_tokens(liste_tok[i]))


sans_ponc_liste = []
for i in range (len(liste_tok)):
    tokens_large_liste.append(supp_ponc(liste_tok[i]))

list_sansStopWords = []
for i in range (len(liste_tok)):
    list_sansStopWords.append(remove_stopwords(tokens_large_liste[i]))

stem_list = []
for i in range (len(list_sansStopWords)):
    stem_list.append(stemming(list_sansStopWords[i]))

lemm_list = []
for i in range (len(list_sansStopWords)):
    lemm_list.append(lemmatizing(list_sansStopWords[i]))



In [ ]:
# Ajout d'une colonne "tokens"
data.insert(loc=3, column="removed_punc", value=liste_tok)
data.insert(loc=4, column="tokens", value=liste_tok)
data.insert(loc=5, column="larger_tokens", value=liste_tok)
data.insert(loc=6, column="clean_tokens", value=list_sansStopWords)
data.insert(loc=7, column="stem_words", value=stem_list)
data.insert(loc=8, column="lemm_words", value=lemm_list)
clean_text = clean(data['stem_words'])
data.insert(loc=9, column="clean_text", value=clean_text)
data.head()

,Unnamed: 0,label,text,removed_punc,tokens,larger_tokens,clean_tokens,stem_words,lemm_words,clean_text
0,0,Psoriasis,I have been experiencing a skin rash on my arm...,"[I, have, been, experiencing, a, skin, rash, o...","[I, have, been, experiencing, a, skin, rash, o...","[I, have, been, experiencing, a, skin, rash, o...","[experiencing, skin, rash, arms, legs, torso, ...","[experienc, skin, rash, arm, leg, torso, past,...","[experiencing, skin, rash, arm, leg, torso, pa...",experienc skin rash arm leg torso past week it...
1,1,Psoriasis,"My skin has been peeling, especially on my kne...","[My, skin, has, been, peeling, especially, on,...","[My, skin, has, been, peeling, especially, on,...","[My, skin, has, been, peeling, especially, on,...","[skin, peeling, especially, knees, elbows, sca...","[skin, peel, especi, knee, elbow, scalp, thi, ...","[skin, peeling, especially, knee, elbow, scalp...",skin peel especi knee elbow scalp thi peel oft...
2,2,Psoriasis,I have been experiencing joint pain in my fing...,"[I, have, been, experiencing, joint, pain, in,...","[I, have, been, experiencing, joint, pain, in,...","[I, have, been, experiencing, joint, pain, in,...","[experiencing, joint, pain, fingers, wrists, k...","[experienc, joint, pain, finger, wrist, knee, ...","[experiencing, joint, pain, finger, wrist, kne...",experienc joint pain finger wrist knee pain of...
3,3,Psoriasis,"There is a silver like dusting on my skin, esp...","[There, is, a, silver, like, dusting, on, my, ...","[There, is, a, silver, like, dusting, on, my, ...","[There, is, a, silver, like, dusting, on, my, ...","[There, silver, like, dusting, skin, especiall...","[there, silver, like, dust, skin, especi, lowe...","[There, silver, like, dusting, skin, especiall...",there silver like dust skin especi lower back ...
4,4,Psoriasis,"My nails have small dents or pits in them, and...","[My, nails, have, small, dents, or, pits, in, ...","[My, nails, have, small, dents, or, pits, in, ...","[My, nails, have, small, dents, or, pits, in, ...","[nails, small, dents, pits, often, feel, infla...","[nail, small, dent, pit, often, feel, inflamma...","[nail, small, dent, pit, often, feel, inflamma...",nail small dent pit often feel inflammatori te...


**Vectorization**

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(data['clean_text']).toarray()

In [ ]:
Y = pd.get_dummies(data["label"]).to_numpy()

**Spliting data into training and testing sets**

In [ ]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=42)

In [ ]:
from keras.layers import Dense, Activation, Dropout
import tensorflow as tf
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(X_train[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(y_train[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = tf.keras.optimizers.legacy.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(
    loss='categorical_crossentropy', 
    optimizer=sgd, 
    metrics=['accuracy'])

#fitting the model
history = model.fit(X_train, y_train, 
    epochs=100, 
    batch_size=8, 
    verbose=1)


Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


135/135 [==============================] - 1s 2ms/step - loss: 3.1728 - accuracy: 0.0574
Epoch 2/100
135/135 [==============================] - 0s 2ms/step - loss: 3.1189 - accuracy: 0.0935
Epoch 3/100
135/135 [==============================] - 0s 2ms/step - loss: 2.9974 - accuracy: 0.1861
Epoch 4/100
135/135 [==============================] - 0s 2ms/step - loss: 2.7407 - accuracy: 0.2639
Epoch 5/100
135/135 [==============================] - 0s 2ms/step - loss: 2.3050 - accuracy: 0.3731
Epoch 6/100
135/135 [==============================] - 0s 2ms/step - loss: 1.8735 - accuracy: 0.4861
Epoch 7/100
135/135 [==============================] - 0s 2ms/step - loss: 1.5290 - accuracy: 0.5769
Epoch 8/100
135/135 [==============================] - 0s 1ms/step - loss: 1.2890 - accuracy: 0.6204
Epoch 9/100
135/135 [==============================] - 0s 2ms/step - loss: 1.0511 - accuracy: 0.6880
Epoch 10/100
135/135 [==============================] - 0s 2ms/step - loss: 0.9099 - accuracy: 0.7380
E

In [ ]:
loss_train, accuracy_train = model.evaluate(X_train, y_train, verbose=0)
loss_test, accuracy_test = model.evaluate(X_test, y_test, verbose=0)

In [ ]:
print("Train accuracy:",accuracy_train*100,"%")
print("Test accuracy:",round(accuracy_test*100,2),"%")

Train accuracy: 100.0 %
Test accuracy: 98.33 %


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)
print(confusion_matrix(y_true, y_pred))
print(classification_report(y_true, y_pred))


**Testing the model**

In [ ]:
# Define the test data
txt = ["My throat is scratchy and I have a persistent cough. I also have a fever and feel fatigued. My body aches and I have a loss of appetite. I have difficulty sleeping due to the discomfort and my breathing feels labored at times."]

# Vextorize the test data
test_features = vectorizer.transform(txt).toarray()

# Use the trained model to predict labels for the test data
predicted_labels = model.predict(test_features)

# Print the predicted labels
predicted_maladie = pd.get_dummies(data["label"]).columns[np.argmax(predicted_labels)]
print(predicted_maladie)

1/1 [==============================] - 0s 15ms/step
Bronchial Asthma


In [ ]:
txt = ["My skin is dry and itchy. I can see some rashes on my skin and my face turns red easily."]
# Vextorize the test data
test_features = vectorizer.transform(txt).toarray()

# Use the trained model to predict labels for the test data
predicted_labels = model.predict(test_features)

# Print the predicted labels
predicted_maladie = pd.get_dummies(data["label"]).columns[np.argmax(predicted_labels)]
print(predicted_maladie)

1/1 [==============================] - 0s 15ms/step
Fungal infection


In [ ]:
model.save('model.h5')